In [72]:
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot
import numpy as np
import pandas as pd

In [2]:
ds = pd.read_csv('/Users/katychow/Projects/quantified_self/Data/data_for_modeling_cleaned_20190204.csv')

In [3]:
ds.head()

,dates,efficiency,asleep_min,awake_min,restless_min,timeInBed,restless_rolling7,awake_rolling7,dayofweek,caffeine_mg,drink_counter
0,2018-10-08,96.0,347.0,2.0,14.0,363.0,0.0,0.0,0,NaN,NaN
1,2018-10-09,93.0,337.0,4.0,21.0,362.0,0.0,0.0,1,NaN,NaN
2,2018-10-10,96.0,417.0,2.0,17.0,437.0,0.0,0.0,2,NaN,NaN
3,2018-10-11,96.0,417.0,2.0,17.0,437.0,0.0,0.0,3,NaN,NaN
4,2018-10-12,96.0,508.0,0.0,24.0,532.0,0.0,0.0,4,NaN,NaN


In [4]:
ds['diff_asleep'] = ds['asleep_min'].diff()

In [10]:
ds.head()

,efficiency,asleep_min,awake_min,restless_min,timeInBed,restless_rolling7,awake_rolling7,dayofweek,caffeine_mg,drink_counter,diff_asleep
dates,,,,,,,,,,,
2018-10-08,96.0,347.0,2.0,14.0,363.0,0.0,0.0,0,0.0,0.0,0.0
2018-10-09,93.0,337.0,4.0,21.0,362.0,0.0,0.0,1,0.0,0.0,-10.0
2018-10-10,96.0,417.0,2.0,17.0,437.0,0.0,0.0,2,0.0,0.0,80.0
2018-10-11,96.0,417.0,2.0,17.0,437.0,0.0,0.0,3,0.0,0.0,0.0
2018-10-12,96.0,508.0,0.0,24.0,532.0,0.0,0.0,4,0.0,0.0,91.0


In [6]:
ds.fillna(0,inplace=True)

In [9]:
ds.set_index('dates', inplace = True)

In [12]:
# need to remove asleep_min, efficiency to predict diff_sleep
ds.drop(columns = ['efficiency','asleep_min'],inplace = True)

In [13]:
ds.shape

(121, 9)

In [14]:
ds.columns

Index(['awake_min', 'restless_min', 'timeInBed', 'restless_rolling7',
       'awake_rolling7', 'dayofweek', 'caffeine_mg', 'drink_counter',
       'diff_asleep'],
      dtype='object')

In [37]:
X = ds[['awake_min', 'restless_min', 'timeInBed', 'restless_rolling7','awake_rolling7', 'dayofweek', 'caffeine_mg', 'drink_counter']]
y = ds['diff_asleep']

In [53]:
y.values

array([   0.,  -10.,   80.,    0.,   91.,  -91., -133.,  131.,   13.,
        -11.,    0.,  -18.,   39.,  -14.,   -7.,    0.,    0.,  -91.,
        128.,  -37.,   27.,  -22.,  -14.,   36., -311.,  310.,  -26.,
          0.,   -8.,    8.,  -65.,    0.,  115., -147.,   97.,    0.,
         39.,  -39.,    9.,   -4.,  -59.,   57.,   -3.,    0.,    0.,
          0.,  -44.,  186., -142., -338.,  338.,    0.,   12.,    5.,
          2.,  -25.,    6.,   64.,  -64.,  -21.,   21., -361.,  339.,
       -339.,  339.,    7.,   15.,  -80.,   80.,   -1.,  -12.,   55.,
        -50., -324.,  229.,   80.,   23.,    0.,   71., -185.,  114.,
          0.,   71.,  -27.,  -44.,    0.,    0.,    0.,    0.,  -38.,
         42.,   -2.,   18.,  -20.,   13., -366.,  374., -374.,  374.,
         97.,  -48.,  -69.,   65.,   31., -124.,   43.,  -28.,  -47.,
         99.,  -75.,   35.,   71., -117.,  123., -405.,  328.,    0.,
         48., -102.,   54.,  260.])

In [38]:
X.shape, y.shape

((121, 8), (121,))

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [46]:
np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)

array([[[  2.        ],
        [ 28.        ],
        [518.        ],
        [ 19.28571429],
        [  2.85714286],
        [  2.        ],
        [  0.        ],
        [  0.        ]],

       [[  2.        ],
        [  6.        ],
        [416.        ],
        [ 13.42857143],
        [  0.85714286],
        [  1.        ],
        [  0.        ],
        [  0.        ]],

       [[  2.        ],
        [  1.        ],
        [ 67.        ],
        [ 14.57142857],
        [  1.42857143],
        [  5.        ],
        [  0.        ],
        [  0.        ]],

       [[  2.        ],
        [ 17.        ],
        [437.        ],
        [ 14.        ],
        [  1.14285714],
        [  5.        ],
        [  0.        ],
        [  0.        ]],

       [[  4.        ],
        [ 11.        ],
        [503.        ],
        [ 23.        ],
        [  2.14285714],
        [  4.        ],
        [247.5       ],
        [  3.        ]],

       [[  2.        ],
      

In [47]:
X_train_reshaped = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_reshaped = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

In [61]:
X_train_reshaped.shape

(90, 8, 1)

In [79]:
early_stop = EarlyStopping(monitor='mae', min_delta=0)
callbacks_list = [early_stop]

In [80]:
model = Sequential()
model.add(LSTM(8, input_shape=(X_train_reshaped.shape[1],X_train_reshaped.shape[2])))
# model.add(Dense(test.shape[0],activation = 'sigmoid'))
model.add(Dense(1))

In [81]:
model.compile(loss='mean_squared_error',optimizer='adam',metrics=['mae'])

In [82]:
history = model.fit(X_train_reshaped, y_train.values,epochs =100, verbose=2, callbacks=callbacks_list)

Epoch 1/100
 - 1s - loss: 17462.0710 - mean_absolute_error: 79.8473
Epoch 2/100
 - 0s - loss: 17461.6590 - mean_absolute_error: 79.8475
Epoch 3/100
 - 0s - loss: 17461.1578 - mean_absolute_error: 79.8475
Epoch 4/100
 - 0s - loss: 17460.8391 - mean_absolute_error: 79.8482
Epoch 5/100
 - 0s - loss: 17460.5984 - mean_absolute_error: 79.8490
Epoch 6/100
 - 0s - loss: 17460.1821 - mean_absolute_error: 79.8498
Epoch 7/100
 - 0s - loss: 17460.0814 - mean_absolute_error: 79.8514
Epoch 8/100
 - 0s - loss: 17459.8982 - mean_absolute_error: 79.8529
Epoch 9/100
 - 0s - loss: 17459.5690 - mean_absolute_error: 79.8545
Epoch 10/100


/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `mae` which is not available. Available metrics are: loss,mean_absolute_error
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 0s - loss: 17459.1441 - mean_absolute_error: 79.8552
Epoch 11/100
 - 0s - loss: 17458.8890 - mean_absolute_error: 79.8555
Epoch 12/100
 - 0s - loss: 17458.6132 - mean_absolute_error: 79.8562
Epoch 13/100
 - 0s - loss: 17458.2735 - mean_absolute_error: 79.8568
Epoch 14/100
 - 0s - loss: 17458.2118 - mean_absolute_error: 79.8584
Epoch 15/100
 - 0s - loss: 17457.6857 - mean_absolute_error: 79.8587
Epoch 16/100
 - 0s - loss: 17457.4518 - mean_absolute_error: 79.8603
Epoch 17/100
 - 0s - loss: 17457.2882 - mean_absolute_error: 79.8613
Epoch 18/100
 - 0s - loss: 17457.1694 - mean_absolute_error: 79.8633
Epoch 19/100
 - 0s - loss: 17456.7007 - mean_absolute_error: 79.8640
Epoch 20/100
 - 0s - loss: 17456.4678 - mean_absolute_error: 79.8647
Epoch 21/100
 - 0s - loss: 17456.1159 - mean_absolute_error: 79.8652
Epoch 22/100
 - 0s - loss: 17456.0318 - mean_absolute_error: 79.8664
Epoch 23/100
 - 0s - loss: 17455.5862 - mean_absolute_error: 79.8674
Epoch 24/100
 - 0s - loss: 17455.2340 - mean_ab